In [ ]:
# src/preprocess.py
import cv2, numpy as np
from mtcnn import MTCNN
from pathlib import Path

detector = MTCNN()

def crop_face(img: np.ndarray, margin=40):
    """Return 256x256 face-centered crop or None."""
    rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    faces = detector.detect_faces(rgb)
    if not faces:
        return None
    x, y, w, h = faces[0]['box']
    x1 = max(x - margin, 0)
    y1 = max(y - margin, 0)
    x2 = x + w + margin
    y2 = y + h + margin
    face = rgb[y1:y2, x1:x2]
    face = cv2.resize(face, (256, 256))
    return face

def preprocess_folder(src: str, dst: str):
    Path(dst).mkdir(parents=True, exist_ok=True)
    for f in Path(src).glob("*.jpg"):
        img = cv2.imread(str(f))
        cropped = crop_face(img)
        if cropped is not None:
            cv2.imwrite(f"{dst}/{f.name}", cv2.cvtColor(cropped, cv2.COLOR_RGB2BGR))

In [ ]:
# src/model.py
from tensorflow.keras import layers, models, applications

def build_xception():
    base = applications.Xception(
        weights='imagenet',
        include_top=False,
        input_shape=(256, 256, 3),
        pooling='avg'
    )
    base.trainable = True  # fine-tune all layers

    inputs = layers.Input((256, 256, 3))
    x = applications.xception.preprocess_input(inputs)
    x = base(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(2, activation='softmax', dtype='float32')(x)
    model = models.Model(inputs, outputs)
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy', 'AUC']
    )
    return model

In [ ]:
# src/train.py
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from src.model import build_xception
import os

IMG_SIZE = (256, 256)
BATCH = 32
EPOCHS = 50

train_gen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_gen = ImageDataGenerator()

train = train_gen.flow_from_directory(
    'data/train', target_size=IMG_SIZE, batch_size=BATCH,
    class_mode='categorical', shuffle=True)

val = val_gen.flow_from_directory(
    'data/val', target_size=IMG_SIZE, batch_size=BATCH,
    class_mode='categorical', shuffle=False)

model = build_xception()

callbacks = [
    ModelCheckpoint('models/best_xception.h5', save_best_only=True, monitor='val_auc', mode='max'),
    EarlyStopping(monitor='val_auc', patience=8, restore_best_weights=True, mode='max'),
    ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=4)
]

model.fit(train, validation_data=val, epochs=EPOCHS, callbacks=callbacks)

In [ ]:
# src/evaluate.py
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import matplotlib.pyplot as plt

model = load_model('models/best_xception.h5')
test_gen = ImageDataGenerator().flow_from_directory(
    'data/test', target_size=(256,256), batch_size=32,
    class_mode='categorical', shuffle=False)

y_prob = model.predict(test_gen)
y_pred = np.argmax(y_prob, axis=1)
y_true = test_gen.classes

print(classification_report(y_true, y_pred, target_names=['Real','Fake']))
print("AUC:", roc_auc_score(y_true, y_prob[:,1]))

# Confusion matrix plot
import seaborn as sns
cm = confusion_matrix(y_true, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Real','Fake'], yticklabels=['Real','Fake'])
plt.xlabel('Predicted'); plt.ylabel('True'); plt.show()

In [ ]:
# src/inference.py
from tensorflow.keras.preprocessing import image
import numpy as np
from tensorflow.keras.models import load_model

model = load_model('models/best_xception.h5')

def predict_image(img_path: str, threshold: float = 0.5) -> str:
    img = image.load_img(img_path, target_size=(256, 256))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = x / 255.0
    prob_fake = float(model.predict(x, verbose=0)[0][1])
    label = "Fake" if prob_fake >= threshold else "Real"
    return f"{label} ({prob_fake*100:.1f}% Fake)"

In [ ]:
# src/app.py
import gradio as gr
from src.inference import predict_image
from PIL import Image

def gradio_predict(img: Image.Image):
    img.save("temp.jpg")
    return predict_image("temp.jpg")

gr.Interface(
    gradio_predict,
    gr.Image(type="pil"),
    gr.Textbox(),
    title="Deepfake Image Detector",
    description="Upload any face → instantly know if it’s **Real** or **AI-generated**.",
    examples=[
        ["examples/real_01.jpg"],
        ["examples/fake_01.jpg"]
    ]
).launch(share=True)